
Обработка естественного языка одно из востребованных направлений машинного обучения, которое постоянно развивается. В 2018 году компания Google представила новую модель - BERT, сделавшую прорыв в области обработки естественного языка. Несмотря на то, что сейчас у BERT много [конкурентов](https://towardsdatascience.com/bert-roberta-distilbert-xlnet-which-one-to-use-3d5ab82ba5f8), включая модификации классической модели (RoBERTa, DistilBERT и др.) так и совершенно новые (например, XLNet), BERT всё ещё остается в [топе](https://www.topbots.com/leading-nlp-language-models-2020) nlp-моделей. 

Данная модель применяется во многих задачах: классификации текстов, генерация текста, суммаризация текста и т.д.
Но ее также можно использовать для получения векторных представлений текста - эмбеддингов. Они могут использоваться, например, для последующей кластеризации или в качестве исходных данных для других моделей. 

В этой статье мы рассмотрим применение эмбеддингов, полученных из BERT, для последующего тематического моделирования.

В качестве источника данных будем использовать [архив](https://github.com/yutkin/Lenta.Ru-News-Dataset) с новостями Lenta.ru. Для реализации модели BERT будем использовать библиотеку transformers.

Скачаем и распакуем архив с новостями и предобученную на русских текстах модель BERT с проекта deeppavlov

In [1]:
!gdown https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2
!gdown http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
!bzip2 -d lenta-ru-news.csv.bz2
!tar -xzf /content/rubert_cased_L-12_H-768_A-12_pt.tar.gz

Downloading...
From: https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2
To: /content/lenta-ru-news.csv.bz2
100% 346M/346M [00:02<00:00, 167MB/s]
Downloading...
From: http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
To: /content/rubert_cased_L-12_H-768_A-12_pt.tar.gz
100% 662M/662M [00:22<00:00, 29.8MB/s]


Импортируем нужные библиотеки

In [ ]:
!pip install transformers
!pip install pyyaml==5.4.1

import numpy as np
import pandas as pd
import torch
import os
import json

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer

В распакованном архиве хранится csv-файл с информацией о статьях. Почти для каждой статьи указан заголовок, тематика, тег и дата публикации. 

Посмотрим список тем.

In [ ]:
df = pd.read_csv('/content/lenta-ru-news.csv', low_memory=False)
df['topic'].unique()
# 'Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
# 'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
# 'Наука и техника', 'Бывший СССР', nan, 'Дом', 'Сочи', 'ЧМ-2014',
# 'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
# 'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'

Выберем десять тысяч статей из темы "Наука и техника".

In [3]:
corpus = df.loc[df['topic']=='Наука и техника', 'text'].to_numpy()
corpus = corpus[:10000]

Посмотрим пример текста:

In [ ]:
corpus[0]
# Американские ученые в ближайшее время отправят на орбиту спутник, 
# который проверит два фундаментальных предположения, выдвинутых Альбертом Эйнштейном...

В распакованной папке с моделью есть файл `bert_config.json`. 
Но метод `from_pretrained` ожидает наличие в папке файла `config.json`, а не `bert_config.json`, добавим недостающий файл. Если бы мы использовали обобщенные классы AutoModel и AutoTokenize, то пришлось бы в файл `config.json` добавлять строчку `"model_type": "bert"`, т.к изначально эти классы ничего не знают о типе используемой модели.

In [5]:
with open("/content/rubert_cased_L-12_H-768_A-12_pt/bert_config.json", "r") as read_file, open("/content/rubert_cased_L-12_H-768_A-12_pt/config.json", "w") as conf:
    file = json.load(read_file)
    conf.write(json.dumps(file))
!rm /content/rubert_cased_L-12_H-768_A-12_pt/bert_config.json

В BERT подается на вход не сам текст, а токены. Поэтому вместе со скачанной моделью в папке идет готовый токенизатор.

Подгрузим модель и токенизатор из распакованной папки с помощью метода `from_pretrained`. Для этого нужно просто указать путь к папке. При инициализации модели укажем, чтобы выводились промежуточные состояния.

На самом деле модель можно подгрузить с сайта [Hugging Face](https://huggingface.co/models), просто указав ее корректное название в методе `from_pretrained`. Но вариант с предварительным скачиванием пригодится, например, когда отсутствует доступ к интернету.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('rubert_cased_L-12_H-768_A-12_pt')
model = BertModel.from_pretrained('rubert_cased_L-12_H-768_A-12_pt', output_hidden_states = True)

Далее воспользуемся вспомогательными классами `Dataset` и `Dataloader` из `torch.utils`. Создадим класс `CustomDataset` на основе импортированного класса `Dataset`. Для корректной работы переопределим в нем методы `__len__` и `__getitem__`.

Добавим метод `tokenize`, внутри которого к тексту будет применяться инициализированный ранее `tokenizer`. Формат выходных данных будет `torch.tensor`. Поставим максимальную длину токенизированного текста на 150 токенов.

In [5]:
class CustomDataset(Dataset):
    
    def __init__(self, X):
        self.text = X

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=150)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1) for k, v in output.items()}


eval_ds = CustomDataset(corpus)
eval_dataloader = DataLoader(eval_ds, batch_size=10)

Для получения эмбеддингов, воспользуемся функцией из этой [статьи](https://medium.com/swlh/transformer-based-sentence-embeddings-cd0935b3b1e0). В ней берется выход из последнего слоя и усредняется по каждому значению. Про значения, получаемые на выходе из BERT, можно почитать [здесь](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/).

In [6]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

Переводим модель в состояние валидации и отключаем подсчет градиента, а также переносим на графический ускоритель, если он доступен:

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()

embeddings = torch.Tensor().to(device)

with torch.no_grad():
    for n_batch, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        embeddings = torch.cat([embeddings, mean_pooling(outputs, batch['attention_mask'])])
    embeddings = embeddings.cpu().numpy()

100%|██████████| 1000/1000 [01:28<00:00, 11.29it/s]


Алгоритмы кластеризации времязатратны, поэтому, чтобы сократить длительность рассчетов, уменьшим размерность эмбеддингов с 768 до 15. Для этого применим классический метод уменьшения размерности - "Метод главных компонент", реализованный в `sklearn`.

In [8]:
pca = PCA(n_components=15, random_state=42)
emb_15d = pca.fit_transform(embeddings)

Далее запускаем алгоритм кластеризации. Число кластеров будет выбираться на основе косинусового расстояния. Пороговое расстояние `distance_threshold`, после которого кластеры считаются одним, является гиперпараметром, и с ним можно поэкспериментировать.

In [9]:
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.6, affinity='cosine', linkage='average').fit(emb_15d)

Чтобы визуализировать результат на графике, уменьшим размерность до двух.

In [10]:
pca = PCA(n_components=2, random_state=42)
emb_2d = pd.DataFrame(pca.fit_transform(embeddings), columns=['x1', 'x2'])
emb_2d['label'] = clustering.labels_
emb_2d['label'].nunique() #39

40

У нас получилось 40 разных кластеров.

Построим интерактивный график с помощью библиотеки `plotly` для быстрой навигации по получившимся кластерам.

In [11]:
import plotly.express as px
fig = px.scatter(emb_2d, x='x1', y='x2', color='label', width=800, height=600)
fig.show()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Данный график иллюстрирует распределение кластеров на двумерном пространстве. Для интерпретации получившихся результатов необходимо более детально рассмотреть каждый кластер. Одним из подходов является составление словаря частотности слов. Более подробно об этом можно почитать [здесь](https://medium.com/nuances-of-programming/тематическое-моделирование-с-помощью-bert-97c9a0be689d).

Мы лишь посмотрим несколько первых предложений внутри кластеров и оценим их схожесть. Рассмотрим кластер слева под номером 0:

In [12]:
def show_examples(cluster, n):
    for i in range(n):
        print(i, corpus[emb_2d['label'] == cluster][i].split('.')[0])
show_examples(cluster=0, n=3)

# 0 Доплеровский радар в армии США будет использоваться не только для составления метеорологических карт, 
#   но и для раннего предупреждения о биологической или химической атаке с воздуха
# 1 Военно-космические силы США приняли на вооружение новую систему,
#   предназначенную для глушения космических спутников, сообщает агентство Reuters
# 2 На авиабазе ВВС США "Эдвардс" в Калифорнии проведены первые успешные испытания боевого лазера воздушного базирования

0 Доплеровский радар в армии США будет использоваться не только для составления метеорологических карт, но и для раннего предупреждения о биологической или химической атаке с воздуха
1 Военно-космические силы США приняли на вооружение новую систему, предназначенную для глушения космических спутников, сообщает агентство Reuters
2 На авиабазе ВВС США "Эдвардс" в Калифорнии проведены первые успешные испытания боевого лазера воздушного базирования


Вероятнее всего, в этом кластере говорится о военных достижениях США.

Для сравнения посмотрим кластер 6 сверху:

In [ ]:
show_examples(cluster=6, n=3)

# 0 Используя преимущества технологии Blu-ray (синий лазер, в отличие от красного, применяемого в CD и DVD-приводах), 
#   позволяющей создавать сверхтонкие носители информации, корпорация Sony и компания Toppan Printing разработали "бумажный" диск,
#   на который можно записать 25 гигабайт видео
# 1 Американская компания Microvision создала лазерную технологию, 
#   которая позволит человеку видеть дополнительное изображение - помимо той картинки, которую он получает при помощи обычного зрения
# 2 Автор операционной системы Linux Линус Торвальдс (Linus Torvalds) предлагает создать новую систему регистрации изменений, 
#   вносимых в операционную системы, с целью предотвращения любых обвинений в нарушении авторских прав, сообщает Siliconvalley

В этом кластере, вероятнее всего, говорится о средствах передачи информации.

Таким образом, мы рассмотрели, как использовать модель BERT для получения эмеддингов, а также коснулись варианта их дальнейшего применения.